# Get error for all runs

* List files
* Read each file
* Get RMSE for each variant in the file
* Rewrite file with this change


In [1]:
from os import walk
import json
from sklearn.metrics import mean_squared_error


with open('./h2_fci.json', 'r') as reader:
    # Read & print the entire file
    data = reader.read()
    fci_h2 = json.loads(data)  

fci_energies = fci_h2["energies"]

f = []
for (dirpath, dirnames, filenames) in walk('./exp_runs'):
    f.extend(filenames)
    break

print(len(f))

50


In [2]:
all_data = []

for file_name in f:
    with open(f'exp_runs/{file_name}', 'r') as reader:
        # Read & print the entire file
        data = reader.read()
        all_data.append(json.loads(data))

In [3]:
optimizers = ["gradient_descent", "spsa", "adam", "demon_adam"]
train_types= ['meta', 'opt_meta', 'vqe']

In [4]:
all_data[0]['results']["adam"].keys()

dict_keys(['meta', 'opt_meta', 'vqe'])

In [5]:
for data_entry in all_data:
    for opt in optimizers:
        for result_type in train_types:
            if result_type == 'meta':
                data_entry['results'][opt][result_type]['rmse_error'] = mean_squared_error(fci_energies, data_entry['results'][opt][result_type]["energies_after_meta"], squared=False)
            else:
                data_entry['results'][opt][result_type]['rmse_error'] = mean_squared_error(fci_energies, [energy[-1] for energy in data_entry['results'][opt][result_type]["energies"]], squared=False)     

In [6]:
for file_name,data_entry in zip(f,all_data):
    with open(f'exp_runs/{file_name}', 'w') as writer:
        writer.write(json.dumps(data_entry))